# 调包区

In [2]:
from tqdm.notebook import tqdm
import random
import pandas as pd
import numpy as np
import math
from matplotlib import pyplot as plt
import gc

# 警告忽略
import warnings
warnings.filterwarnings("ignore")

# matplotlib字体设置
plt.rcParams["font.family"] = "Songti SC"
plt.rcParams["axes.unicode_minus"] = False

# matplotlib警告忽略
pd.plotting.register_matplotlib_converters()


# 观看Dataframe长度
pd.set_option('display.max_rows', 10)
pd.set_option('display.max_columns', None)
# 浮点数位长度
pd.set_option('display.precision',5)

# 显示多个结果
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = 'all' # ['all', 'last', 'last_expr', 'none', 'last_expr_or_assign']

In [16]:
from datetime import datetime
import jqdatasdk as jq

# 登陆
jq.auth('18689717301', 'Zz.7895123')
#jq.auth('13282159964', '159964')


# 查询使用次数
count=jq.get_query_count()
print("当日可调用数据总条数为：",count['total'])
print("当日剩余可调用条数为：",count['spare'])


提示：当前环境 pandas 版本高于 0.25，get_price 与 get_fundamentals_continuously 接口 panel 参数将固定为 False
注意：0.25 以上版本 pandas 不支持 panel，如使用该数据结构和相关函数请注意修改
auth success 
当日可调用数据总条数为： 1000000
当日剩余可调用条数为： 989824


# 打开文档

In [11]:
path = './temdata'
df = pd.read_csv(f'{path}/stock.csv',index_col=0)
df

,id,code,pe_ratio,turnover_ratio,pb_ratio,ps_ratio,pcf_ratio,capitalization,market_cap,circulating_cap,circulating_market_cap,day,pe_ratio_lyr
0,62085031,000001.XSHE,13.9148,NaN,1.2994,2.4763,2.7943,1.94059e+06,3753.1045,1.94058e+06,3753.0725,2021-01-01,13.3112
1,62083452,000002.XSHE,8.2340,NaN,1.6451,0.8650,4.6903,1.16177e+06,3334.2891,9.71432e+05,2788.0083,2021-01-01,8.5776
2,62083558,000004.XSHE,48.4751,NaN,2.3569,23.2341,31.4974,1.65053e+04,34.1659,8.39187e+03,17.3712,2021-01-01,1102.3503
3,62083192,000005.XSHE,26.1115,NaN,1.6504,13.3094,-9379.1064,1.05854e+05,26.7810,1.05795e+05,26.7660,2021-01-01,15.3307
4,62084251,000006.XSHE,8.8088,NaN,1.0805,2.2464,-5.7974,1.35000e+05,75.7347,1.34831e+05,75.6401,2021-01-01,9.4531
...,...,...,...,...,...,...,...,...,...,...,...,...,...
4135,62083432,688699.XSHG,50.1476,NaN,3.9611,9.0038,-94.0660,7.43680e+03,41.0288,1.69143e+03,9.3316,2021-01-01,50.8257
4136,62086029,688777.XSHG,126.4118,NaN,13.1172,16.8728,1085.4418,4.91290e+04,492.2726,3.90329e+03,39.1109,2021-01-01,134.6860
4137,62083247,688788.XSHG,41.4656,NaN,3.7112,12.6815,-480.1616,7.55342e+03,100.6871,1.66314e+03,22.1696,2021-01-01,46.1936
4138,62086366,688981.XSHG,119.3556,NaN,4.4766,16.5714,8.8277,7.70351e+05,4448.7749,1.04023e+05,600.7335,2021-01-01,248.0134


In [109]:
# 定义初始状态
import collections
port = collections.defaultdict(lambda : 0)

# 定义初始资金
port['cash'] = 100000

# 购买股票

In [123]:
# 买入股票
def buy_stock(port,code_name,buy_count,buy_data,info=True):
    # 定位股票
    df = jq.get_price(code_name, start_date=buy_data, end_date=buy_data)

    # 得到价格
    price = df['open']

    if price.empty:
        print('股票不存在or不再交易日内')
        return port

    # 制定买入金额
    price  = price[0]*buy_count
    
    if port['cash'] < price:
        print('你的钱不购买股票，本次操作跳过')
        return port
        

    # 更新数据
    port[code_name]+=buy_count # 更新购买的数量
    
    port['cash']-=price # 更新现金
    
    if info==True:
        cash = port['cash']
        print(f'日期:{buy_data},买入:{code_name},数量:{buy_count},金额:{price},剩余现金:{cash}')
    return port
    
    
buy_data = "2021-01-04"
buy_count = 10000
code_name = '000001.XSHE'
port = port

port = buy_stock(port,code_name,buy_count,buy_data)
port

你的钱不购买股票，本次操作跳过


defaultdict(<function __main__.<lambda>()>,
            {'cash': 90634.0,
             '000001.XSHE': 0,
             '000002.XSHE': 100,
             '000004.XSHE': 500})

日期:2021-01-04,买入:000001.XSHE,数量:500,金额:9550.0,剩余现金:66781.0
日期:2021-01-05,买入:000002.XSHE,数量:600,金额:16668.0,剩余现金:50113.0
日期:2021-01-06,买入:000004.XSHE,数量:700,金额:14511.0,剩余现金:35602.0


defaultdict(<function __main__.<lambda>()>,
            {'cash': 35602.0,
             '000001.XSHE': 1000,
             '000002.XSHE': 700,
             '000004.XSHE': 1400})

# 卖出股票

In [114]:
# 卖出股票
def sell_stock(port,code_name,sell_count,sell_data,info=True):
    # 定位股票
    df = jq.get_price(code_name, start_date=sell_data, end_date=sell_data)

    # 得到价格
    price = df['open']

    if price.empty:
        print('股票不存在or不再交易日内')
        return port

    # 检查持仓数量(如果持仓数少于卖出数，则变为全部卖出)
    if port[code_name]<sell_count:
        sell_count = port[code_name]

    # 计算卖出的金额
    price  = price[0] * sell_count

    # 更新数据
    port[code_name]-=sell_count # 更新购买的数量
    port['cash']+=price # 更新现金
    
    if info==True:
        cash = port['cash']
        print(f'日期:{buy_data},卖出:{code_name},数量:{buy_count},金额:{price},剩余现金:{cash}')
    
    return port

sell_data = "2021-01-20"
sell_count = 500
code_name = '000002.XSHE'
port = port

port = sell_stock(port,code_name,sell_count,sell_data,info=True)
port

日期:2021-01-06,卖出:000002.XSHE,数量:700,金额:15535.0,剩余现金:85881.0


defaultdict(<function __main__.<lambda>()>,
            {'cash': 85881.0,
             '000001.XSHE': 0,
             '000002.XSHE': 100,
             '000004.XSHE': 700})

In [122]:
# 批量卖出测试
code_list = ['000001.XSHE','000002.XSHE','000004.XSHE']
date_list = ["2021-01-15","2021-01-14","2021-01-15"]
buy_list = [1000,300,300]
info_list = zip(code_list,date_list,buy_list)

# 批量
for code_name,buy_data,buy_count in info_list:
    port = sell_stock(port,code_name,buy_count,buy_data)
    
port
    


日期:2021-01-15,卖出:000001.XSHE,数量:1000,金额:14700.0,剩余现金:76297.0
日期:2021-01-14,卖出:000002.XSHE,数量:300,金额:8979.0,剩余现金:85276.0
日期:2021-01-15,卖出:000004.XSHE,数量:300,金额:5358.0,剩余现金:90634.0


defaultdict(<function __main__.<lambda>()>,
            {'cash': 90634.0,
             '000001.XSHE': 0,
             '000002.XSHE': 100,
             '000004.XSHE': 500})

# 查询当前持仓价值

In [125]:
def check_stock(port,today_data):
    value = 0
    for code,count in port.items():
        if code == 'cash':
            value+=count
        else:
            # 定位股票
            df = jq.get_price(code, start_date=today_data, end_date=today_data)

            # 得到价格
            price = df['open']

            if price.empty:
                print('股票不存在or不再交易日内,以下价值仅包含现金')
                #return port
            else:
                value_ = price[0] * count

                value+=value_

    print('当前持仓的价值为:',value)
    
today_data = "2021-01-15"
port = port
port = check_stock(port,today_data)

port

当前持仓的价值为: 102565.0
